# Pythonプログラミング入門 第6回
関数プログラミングについて説明します

参考
- https://docs.python.org/ja/3/howto/functional.html
- https://docs.python.org/ja/3/library/functions.html
- https://docs.python.org/ja/3/library/itertools.html
- https://docs.python.org/ja/3/library/functools.html

# 関数プログラミング

**関数プログラミング**とは、プログラムを（数学的な）関数の合成で記述するプログラミングスタイルです。
処理を操作列と考えて命令的に記述するのではなく、処理をデータ変換を行う関数に分解して記述します。
これをPythonで行うときに重要になるのは、高階関数とイテレータです。
したがって、Pythonにおける関数プログラミングとは、高階関数とイテレータを使いこなすことだと考えても、ほぼ差し支えありません。

## 高階関数

**高階関数** （**higher-order function**）とは、値として関数を受け取ったり返したりする関数のことです。
Pythonにおける関数はオブジェクトなので、定義した関数をそのまま渡したり返したりすることができます。

In [1]:
def inc(x):
    return x+1

def twice(f, x):
    return f(f(x))

def genfunc():
    return inc

twice(genfunc(), 0)

2

ここで、`twice()`は関数を受け取り、`genfunc()`は関数を返しているので、どちらも高階関数です。

組込み関数などのよく使われる関数には、関数を受け取る高階関数が多いです。
そのような高階関数を使うときには、上に示した`inc()`のように、小さい関数を渡したくなることがよくあります。
この時に便利なのが、**ラムダ式**（または**無名関数**）です。例えば、
```Python
lambda x: x+1
```
は、`inc()`と等価な関数オブジェクトと返します。一般に、
```Python
f = lambda 引数: 式
```
は
```Python
def f(引数):
    return 式
```
と同等です。

ラムダ式は、`def`記法による関数定義に比べて記述に制限が加わりますが、関数呼出しの引数の位置に関数定義を記述できるという利点があります。
例えば、`twice(inc, 0)`の代わりに`twice(lambda x: x+1, 0)`と呼び出すなら、わざわざ`inc()`を定義しなくて済みます。
このように、ラムダ式を有効活用すると、全体のコードが簡潔で読みやすくなります。

### sorted
2-2で、整列（ソート）されたリストを返す関数`sorted()`を導入しました。

In [2]:
sorted([1,3,-2,0])

[-2, 0, 1, 3]

実は、`sorted()`は`key`引数に関数を取れる高階関数です。
`key`引数は、各要素を比較に使われる値に変換する関数を取ります。
例えば、絶対値の昇順で整列したい場合、絶対値関数`abs()`を`key`引数に渡せばよいです。

In [3]:
sorted([1,3,-2,0], key=abs)

[0, 1, -2, 3]

#### 練習
文字列のキーと数値の値のペアのリスト`ls`があるとする。例えば、`ls = [('A', 1), ('B', 3), ('C', -1), ('D', 0)]`。
このリスト`ls`を、値の降順で整列するように、`sorted()`を呼び出せ。

In [5]:
#自分で新しくセルを作った。以下の練習も同様
ls = [('A', 1), ('B', 3), ('C', -1), ('D', 0)]
sorted(ls, key=lambda x: -x[1])

[('B', 3), ('A', 1), ('D', 0), ('C', -1)]

### max, min

組込み関数`max()`と`min()`は、それぞれ最大の要素と最小の要素を返す関数です。

In [6]:
max([1,3,-2,0])

3

In [7]:
min([1,3,-2,0])

-2

`sorted()`と同様に、どちらも`key`引数に、比較に使われる値に変換する関数を取れます。
したがって、例えば`abs()`を渡せば、絶対値が最大と最小となる要素を返します。

In [8]:
max([1,-3,-2,0], key=abs)

-3

In [9]:
min([1,-3,-2,0], key=abs)

0

#### 練習
リスト（例えば`[1,3,-2,0]`）の最小の要素を返すように、`max()`を用いよ。
ただし、リストの各要素は数値だと仮定して良い。

In [10]:
max([1,3,-2,0], key=lambda x: -x)

-2

### ▲reduce

3-3や3-4で、組込み関数の`sum()`を紹介しました。
これは総和を返す組込み関数でした。

In [11]:
sum([-1,-3,2,4])

2

総和があるならば、総乗を取るような組込み関数があるかというと、ありません。
しかし、`functools`モジュールには、総和や総乗を一般化した関数`reduce()`があります。

`reduce()`は、第1引数にとる2引数関数を使って、第2引数を前から順に畳み込む関数です。
前から順に畳み込むとは、具体的には、第1引数が`f`で、第2引数が`[-1,-3,2,4]`のとき、`f(f(f(-1, -3), 2), 4)`という演算です。
したがって、総和も総乗も次のように表現できます。

In [12]:
import functools
functools.reduce(lambda x,y: x+y, [-1,-3,2,4])

2

In [13]:
functools.reduce(lambda x,y: x*y, [-1,-3,2,4])

24

`sum()`の第2引数に初期値を取れるように、`reduce()`も第3引数に初期値を取れます。

In [14]:
sum([-1,-3,2,4], 10)

12

In [15]:
functools.reduce(lambda x,y: x*y, [-1,-3,2,4], 10)

240

初期値は、第2引数の要素とは異なるデータ型を取ることを許されます。
与える関数の第1引数と第2引数も、異なるデータ型を取ることを許されます。
したがって、巧妙に初期値と引数関数を設定することで、様々な計算を`reduce()`で実現できます。

In [16]:
def enumstep(x, y):
    i, ls = x
    ls.append((i,y))
    return (i + 1, ls)
functools.reduce(enumstep, 'ACDB', (0,[]))[1]

[(0, 'A'), (1, 'C'), (2, 'D'), (3, 'B')]

ただし、このように複雑になってくると、素直にfor文で書いた方が見やすくなることも多々あります。
`reduce()`の利用には、バランス感覚が重要です。

## イテレータ

前述の`sorted()`、`min()`、`max()`などは、リストとタプルの両方を同様に渡して処理することができます。
for文で走査（全要素を訪問）するときも、リストとタプルは同様に扱えます。
何故、異なるものを同じように扱えるのでしょうか。
それはイテレータという仕掛けがあるからです。

イテレータとは、コレクション（要素の集まり）を走査するオブジェクトです。
組込み関数`iter()`によって構築し、組込み関数`next()`によって要素を取り出します。

In [17]:
it = iter([1,2]) # [1,2]のイテレータを構築
next(it)

1

In [18]:
next(it)

2

In [19]:
next(it)

StopIteration: 

`next()`は、返す要素がないとき（走査の終了時）に、`StopIteration`という例外を投げます。

イテレータもfor文で反復処理できます。

In [20]:
it = iter([1,2])
for x in it:
    print(x)

1
2


for文では、`StopIteration`を検知して反復を自動的に終了しています。

ここで重要なのは、リストやタプルを含めたコレクションは、全てイテレータを経由して走査するということです。
つまり、リストやタプルなど異なるものから、イテレータという同様に操作できるオブジェクトを構築して利用することで、同じように走査できるようになったわけです。

In [21]:
it = iter((1,2)) # (1,2)のイテレータを構築
for x in it:
    print(x)

1
2


ここで注意すべきことは、イテレータは1回の走査にしか使えない、使い捨てのオブジェクトだということです。
同じコレクションを複数回走査したいときには、走査する度にイテレータを構築する必要があります。

In [22]:
next(it) # (1,2)のイテレータitは走査が終了したまま

StopIteration: 

In [23]:
for x in it:
    print('これは呼び出されない')

ここで憶えておくべきことは、イテレータ自体は元のコレクションをコピーしないということです。
要素を1つ1つ訪問するという反復処理を実現するオブジェクトであり、通常`iter()`や`next()`はコレクションのサイズ（要素数）に依存しないコストで実装されます。
例えば、リストの先頭要素を除いた残りの部分を走査するとき、
```Python
for x in ls[1:]:
    何かの処理

```
と残りの部分をスライスとしてコピーするよりも、
```Python
it = iter(ls)
next(it) # 先頭要素を捨てる
for x in it:
    何かの処理

```
とイテレータで直接走査する方が効率的です。
これは、サイズが小さいコレクションを扱うときには問題になりませんが、大きいものを扱うときには気を付けるべきことです。

ここまで、イテレータの使い方は、`next()`で要素を取り出すか、for文で反復するだけでしたが、実は`sorted()`、`max()`、`min()`などに渡すことができます。

イテレータ`it`の中身を印字して確認したいときには、`print(*it)`と、イテレータを展開して可変長引数として`print()`を呼び出すのが簡潔で便利です。
ただし、中身を確認した後の`it`はもう利用できないこと、大量の要素を生成するイテレータには不向きであることに留意してください。

In [24]:
it = iter(range(4))
next(it) # 先頭の 0 を捨てる
print(*it)

1 2 3


イテレータの定義は、6-3で改めて説明します。

### 練習
与えられたコレクションの先頭要素を除いた残りの部分の最大値を返す関数`tailmax()`を、イテレータを使って、for文を使わずに、上の例に倣って効率的に実装せよ。

In [25]:
def tailmax(xs):
    it = iter(xs)
    next(it) # 先頭要素を捨てる
    return max(it)

print(tailmax([3,-4,2,1]) == 2)
print(tailmax((3,-4,2,1)) == 2)
print(tailmax('ACDC') == 'D')

True
True
True


## イテレータを生成する関数

Pythonの組込み関数や標準ライブラリには、イテレータを返す関数が数多くあります。
その中には、関数を受け取る高階関数もあります。
イテレータを生成・消費する関数の適用に分解してプログラムを記述することで、イテレータを介した関数プログラミングが行えるようになります。

### enumerate
3-2で紹介した組込み関数の`enumerate()`は、実はイテレータを返します。

In [26]:
it = enumerate('ACDB')
print(it) # リストやタプルではない

In [27]:
print(*it)

(0, 'A') (1, 'C') (2, 'D') (3, 'B')


つまり、for文や内包表現に限定されず、イテレータを消費する関数と共に使えます。

そして、`enumerate()`は、コレクションだけでなく、イテレータも渡せます。
つまり、計算結果のイテレータの各要素に番号付けすることにも利用できます。

`enumerate()`の第2引数には番号付けの初期値を渡せます。

In [28]:
print(*enumerate('ACDB',1))

(1, 'A') (2, 'C') (3, 'D') (4, 'B')


`enumerate()`は、番号付けという汎用的なデータ変換を行う関数だったのです。

### map
組込み関数の`map()`は、第1引数に取った関数を、第2引数に取ったコレクションやイテレータの各要素に適用した結果を走査するイテレータを返します。

In [29]:
print(*map(lambda x: x + 1, [1,-3,2,0]))

2 -2 3 1


より正確には、第1引数には、$n$引数関数（$n \ge 1$）を取ることができ、第2引数以降に$n$個のコレクションやイテレータを渡すことができます。この時、一番小さい要素数に合わせて、結果のイテレータは切り詰められます。

In [30]:
# 異なるコレクション/イテレータを受け取れる
print(*map(lambda x,y: x + y, [1,-3,2,0], (4,7,-6,5)))

5 4 -4 5


In [31]:
# 結果のイテレータが切り詰められる
print(*map(lambda x,y: x + y, range(1,10,2), range(1000000)))

1 4 7 10 13


`map()`とラムダ式を組み合わるよりも、3-3で紹介した内包表現（ジェネレータ式を含む）の方が簡潔になることも少なくありません。

In [32]:
print([x + 1 for x in  [1,-3,2,0]])  # リスト内包 

[2, -2, 3, 1]


In [33]:
print(*(x + 1 for x in  [1,-3,2,0])) # ジェネレータ式（イテレータを返す）

2 -2 3 1


一方、既定義の関数を引数に渡すときには、`map()`の方が簡潔になります。
その時々で、内包表記と比べてみて、より分かりやすい方を採用しましょう。

#### 練習
第1引数で与えられた要素数まで、第2引数に与えられたコレクション/イテレータを走査するイテレータを返す関数`take()`を、for文を使わずに、`map()`と`range()`を用いて定義せよ。例えば、`take(2, 'ACDB')`は、`A` `C`を走査するイテレータを返す。

In [37]:
def take(n, xs):
    return map(lambda x, i: x, xs, range(n))

print(*take(2, 'ACDB'))

A C


### zip
組込み関数`zip()`は、`map()`の第1引数の関数が、タプル構築に固定されたものです。

In [38]:
print(*zip(range(1,10,2), range(1000000)))

(1, 0) (3, 1) (5, 2) (7, 3) (9, 4)


上に示したように、結果のイテレータの切り詰めも、同様に行われます。

`zip()`は、`map()`の特殊形でしかないのですが、`map()`を内包表記に書き換えるときや、結果のイテレータをfor文で反復するときに、特に役立ちます。

In [39]:
print([x + y for x, y in zip(range(1,10,2), range(1000000))]) # リスト内包

[1, 4, 7, 10, 13]


In [40]:
print(*(x + y for x, y in zip(range(1,10,2), range(1000000)))) # ジェネレータ式（イテレータを返す）

1 4 7 10 13


In [41]:
for x, y in zip(range(1,10,2), range(1000000)): # for文で反復処理
    print(x + y)

1
4
7
10
13


`map()`とラムダ式を使うか、`zip()`と内包表記を使うかは、より分かりやすい方を、その時々で判断して選択しましょう。

#### 練習
コレクションを取って、隣接要素対のイテレータを返す関数`adjpairs()`を、for文を使わずに`zip()`を使って定義せよ。
例えば、`adjpairs([1,-3,2,0])`は、`(1, -3)` `(-3, 2)` `(2, 0)`を走査するイテレータを返すことになる。

In [42]:
def adjpairs(xs):
    it = iter(xs)
    next(it) # 1つ前にずらす
    return zip(xs, it)

print(*adjpairs([1,-3,2,0]))

(1, -3) (-3, 2) (2, 0)


### filter
組込み関数`filter()`は、第1引数に単項述語（真理値を返す1引数関数）を、第2引数にコレクションもしくはイテレータを取り、その単項述語を真にする要素だけを順に生成するイテレータを返します。

In [43]:
print(*filter(lambda x: x % 2 == 0, range(8)))

0 2 4 6


`filter()`は、制御構造の観点で見ると、continue文によるスキップを含んだfor文に対応します。
continueを含んだfor文を使うときには、代わりに`filter()`を使うことができないか考えてみると良いでしょう。

`map()`と同様に、素直に内包表現に書き換えられます。

In [44]:
print([x for x in range(8) if x % 2 == 0]) # リスト内包

[0, 2, 4, 6]


In [45]:
print(*(x for x in range(8) if x % 2 == 0)) # ジェネレータ式（イテレータを返す）

0 2 4 6


`filter()`とラムダ式を組み合わせるときや、`filter()`と`map()`を組み合わせるときは、内包表記を使った方が簡潔で分かりやすくなることが多いです。

#### 練習
文字列には、文字列のコレクション/イテレータを取る`join()`メソッドがある。
`s.join(ss)`としたとき、`ss`の各要素の文字列を、`s`を間に挟んで連結した結果の文字列を返す。

In [46]:
','.join(['A','B','C','D'])

'A,B,C,D'

In [47]:
','.join(iter('ABCD'))

'A,B,C,D'

この`join()`メソッドと`filter()`を用いて、与えられた文字列の全ての改行`'\n'`と空白`' '`を除去した文字列を返す関数`condense()`を、for文を使わずに定義せよ。例えば`condence('The Zen of Python\n')`は、`'TheZenofPython'`を返す。

In [48]:
def condence(ss):
    return ''.join(filter(lambda s: s not in ('\n',' '), ss))

condence('The Zen of Python\n')

'TheZenofPython'

### takewhile, dropwhile
`itertools`モジュール内の関数`takewhile()`は、`filter()`と同様の引数を取り、単項述語が偽を返すまで走査するイテレータを返します。

In [49]:
import itertools
print(*itertools.takewhile(lambda x: x != 4, range(8)))

0 1 2 3


同じく`itertools`モジュール内の関数`dropwhile()`は、同様の引数を取り、`takewhile()`の残りを走査するイテレータを返します。

In [50]:
print(*itertools.dropwhile(lambda x: x != 4, range(8)))

4 5 6 7


`takewhile()`と`dropwhile()`は、制御構造の観点で見ると、break文による途中脱出を含んだfor文に対応します。
つまり、内部的には反復を途中で打ち切って結果を返しています。
これは、第2引数を走査しきる`filter()`と異なる点であり、実行効率に顕著に影響します。

In [51]:
print(*filter(lambda x: x < 4, range(10000000)))

0 1 2 3


In [52]:
print(*itertools.takewhile(lambda x: x < 4, range(10000000)))

0 1 2 3


同じ結果を返していますが、`filter()`よりも`takewhile()`が速いことが実感できるでしょう。
大量の（もしくは際限なく）要素を生成するイテレータを使うときには、`takewhile()`と`dropwhile()`は、特に役立ちます。

#### 練習
第1引数に整数$k$、第2引数に整列可能なコレクション/イテレータを取り、その中の上位$k$位までを走査するイテレータを返す関数`topk()`を、for文を使わずに`takewhile()`を用いて定義せよ。
ただし、同率順位を考慮するものとする。
例えば、`topk(3, [1,4,3,2,3,4])`は、`4` `4` `3` `3`を走査するイテレータを返すことになる。

In [53]:
def topk(k, xs):
    xs = sorted(xs, reverse=True)
    return itertools.takewhile(lambda x: x >= xs[k-1], xs)
    
print(*topk(3, [1,4,3,2,3,4]))

4 4 3 3


### ▲accumulate
`itertools`モジュール内の関数`accumulate()`は、`sum()`の途中結果をイテレータで返す関数です。

In [54]:
import itertools
print(*itertools.accumulate([1]*8))

1 2 3 4 5 6 7 8


In [55]:
print(*itertools.accumulate(range(8)))

0 1 3 6 10 15 21 28


`func`引数に2引数関数を渡すことができ、この場合は`reduce()`の途中結果をイテレータで返すことに相当します。

In [56]:
print(*itertools.accumulate([1,-5,2,-6,0,3,7], func=max))

1 1 2 2 2 3 7


### reversed
組込み関数`reversed()`は、シーケンス（文字列、リスト、タプルなど）を受け取って、それを逆順に走査するイテレータを返します。

In [57]:
print(*reversed('ABCD'))

D C B A


In [58]:
print(*reversed([0,1,-2,3]))

3 -2 1 0


In [59]:
print(*reversed((0,1,-2,3)))

3 -2 1 0


`reversed()`は、コレクション一般やイテレータを取れないことに留意してください。

#### 練習
与えられたシーケンスを真ん中で折り畳んで閉じ合わせた（zipした）結果をイテレータで返す関数`clamshell()`を、for文を使わずに、`reversed()`と`take()`を使って定義せよ。
ただし、シーケンスの長さが奇数であるとき、中央の要素は結果から除外されるものとする。
例えば、`clamshell('ABCDE')`は、`(A,E)` `(B,D)`を走査するイテレータを返す。

In [61]:
def clamshell(xs):
    return take(len(xs)//2, zip(xs, reversed(xs))) #takeは練習問題で定義した関数 //は整除(商もあまりも整数)

print(*clamshell('ABCDE'))

('A', 'E') ('B', 'D')


### ▲product
`itertools`モジュール内の関数`product()`は、任意個のコレクション/イテレータを取って、それらの直積を取った結果を走査するイテレータを返します。

In [62]:
import itertools
print(*itertools.product('ABCD', range(2)))

('A', 0) ('A', 1) ('B', 0) ('B', 1) ('C', 0) ('C', 1) ('D', 0) ('D', 1)


In [63]:
print(*itertools.product('AB', range(2), 'CD'))

('A', 0, 'C') ('A', 0, 'D') ('A', 1, 'C') ('A', 1, 'D') ('B', 0, 'C') ('B', 0, 'D') ('B', 1, 'C') ('B', 1, 'D')


制御構造の観点で見ると、for文のネストや、for句が連なった内包表記に対応します。
それらを、ネストしない1つのfor文や、for句1つの内包表記で記述するときに役立ちます。

## ▲関数内関数（クロージャ）
関数内で定義された関数（ラムダ式を含む）からは、外側のローカル変数を参照できます。

In [64]:
def outer(x):
    def inner():
        return x
    return inner

f = outer(1)
f()

1

In [65]:
g = outer(2)
g()

2

グローバル変数がそうであるように、外側の関数のローカル変数についても、内側の関数からは再定義が（基本的に）できません。
しかし、外側の関数では再定義できるので、注意が必要です。

In [66]:
def outer(x):
    def inner():
        return x
    x = -x # inner()が参照する変数xを再定義
    return inner

f = outer(1)
f()

-1

それ故に、関数を返す高階関数を記述するときには、変数定義に対してとても慎重になる必要があります。

そういう事情も含めて、関数を返す高階関数を正しく定義するのは難しいので、自分で定義せずに既存の関数を使うだけにするのが無難でしょう。

## 練習の解答

In [ ]:
ls = [('A', 1), ('B', 3), ('C', -1), ('D', 0)]
sorted(ls, key=lambda x: -x[1])

In [ ]:
max([1,3,-2,0], key=lambda x: -x)

In [ ]:
def tailmax(xs):
    it = iter(xs)
    next(it) # 先頭要素を捨てる
    return max(it)

print(tailmax([3,-4,2,1]) == 2)
print(tailmax((3,-4,2,1)) == 2)
print(tailmax('ACDC') == 'D')

In [ ]:
def take(n, xs):
    return map(lambda x, i: x, xs, range(n))

print(*take(2, 'ACDB'))

In [ ]:
def adjpairs(xs):
    it = iter(xs)
    next(it) # 1つ前にずらす
    return zip(xs, it)

print(*adjpairs([1,-3,2,0]))

In [ ]:
def condence(ss):
    return ''.join(filter(lambda s: s not in ('\n',' '), ss))

condence('The Zen of Python\n')

In [ ]:
def topk(k, xs):
    xs = sorted(xs, reverse=True)
    return itertools.takewhile(lambda x: x >= xs[k-1], xs)
    
print(*topk(3, [1,4,3,2,3,4]))

In [ ]:
def clamshell(xs):
    return take(len(xs)//2, zip(xs, reversed(xs)))

print(*clamshell('ABCDE'))